In [1]:
!pip install wordfreq
from wordfreq import word_frequency
import spacy
!pip install openai
from openai import OpenAI
client = OpenAI(api_key='')

In [2]:
def explain_sentences(statement):
    completion = client.chat.completions.create(
        model="gpt-4-1106-preview",
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": f"Explain the following sentences. Only give me your explanation in a paragraph. Do not involve original sentences: {statement}",
            }
        ],
        temperature=0.5,
        top_p=0.5,
    )
    return completion.choices[0].message.content

In [3]:
def examples(sentence):
    completion = client.chat.completions.create(
      model='gpt-4-1106-preview',
      messages=[
        {"role": "system",
         "content": "You are a helpful assistant."},
        {"role": "user",
         "content": f"Add one concise sentence of example if it has, to {sentence} for better understanding. Give me the sentences after adding this example."}
      ],
      temperature=0.5,
      top_p=0.5
    )
    return completion.choices[0].message.content

In [4]:
def check_freq(sentence):

  nlp = spacy.load("en_core_web_sm")
  words = [tok.lemma_ for tok in nlp(sentence) if tok.pos_ not in ["PUNCT", "SPACE"]]

  freq_dict = {}
  for word in words:
    freq = word_frequency(word, 'en')
    freq_dict[word] = freq

  vocab = dict(sorted(freq_dict.items(), key=lambda item: item[1]))
  return vocab

def words_exp(sentence):

  freq_dict = check_freq(sentence)

  explain = dict((k, v) for k, v in freq_dict.items() if v < 1e-4)

  words = list(explain.keys())

  return words

def explain_words(sentence):
  words = words_exp(sentence)
  assis = 'Give me a new verison of sentences which correctly explain these words in simpler sentences or correctly paraphrase with super easy-to-understand words(consider their academic meaning if neccesary):'
  for word in words:
    assis += word
    assis += ', '
  assis += 'inside sentences and only give me the new version of explained sentences.'

  completion = client.chat.completions.create(
      model='gpt-4-1106-preview',
      messages=[
         {"role": "system",
         "content": assis},
        {"role": "user",
         "content": sentence}
      ],
      temperature=0.5,
      top_p=0.5,
  )

  return completion.choices[0].message.content

In [5]:
def simplify_structure(sentence):
  completion = client.chat.completions.create(
      model='gpt-4-1106-preview',
      messages=[
        {"role": "system",
         "content": "Break all sentences into simple sentences without missing any important details and ensure the sentences are readable and coherent. Also do not increase words' complexity, do not give me several points but a coherent paragraph."},
        {"role": "user",
         "content": sentence}
      ],
      temperature=0.5,
      top_p=0.5,
  )
  return completion.choices[0].message.content

In [6]:
def explain(sent):
  tmp = explain_sentences(sent)
  tmp2 = examples(tmp)
  tmp3 = explain_words(tmp2)
  tmp4 = simplify_structure(tmp3)
  result = explain_words(tmp4)
  return result

In [7]:
explain("However, Edwards believes the small changes in solar heating produced by Milankovitch cycles are then amplified by feedback mechanisms on Earth.")

"Edwards thinks that slight shifts in warmth from the sun can change our planet. These shifts come from space patterns called Milankovitch cycles. The Earth's weather system, which includes loops that can make these shifts stronger or weaker, intensifies these changes. These loops are called feedback loops. They can make the sun's warmth changes more or less powerful, causing big weather changes. For instance, when there's less ice, the sea soaks up more warmth. This soaking up can cause even more ice to melt. Edwards hints that we might not see these changes from the sun right away, but they really do shake up our weather. This shake-up happens because of natural processes in the world around us."

In [8]:
explain("Ehrlich's model shows that whilst most of these oscillations cancel each other out, some reinforce one another and become long-lived temperature variations.")

"Ehrlich has a theory about how things work together. Changes in heat or cold happen often. Normally, these changes even out. They act against each other. But sometimes, these changes can join forces. When they join, their effect gets stronger. For example, warm air can mix with a lot of air pressure. This mix can make a really hot time. When this mix happens, the change in heat is big. The change doesn't go away fast. It makes the heat stay different for a long time. Ehrlich's theory points out that in systems that are always changing, certain mixes can cause big and long-lasting changes in heat. These changes are not like the short changes that usually happen."

In [ ]:
import pandas as pd
df = pd.read_csv('data.csv')
df = df.reset_index(drop=True)
original = df[df.columns[0]].tolist()
results = []
for statement in original:
    exp = explain(statement)
    results.append(exp)

df_combined = pd.DataFrame({
    'OriginalStatements': original,
    'ExplainedStatements': results,
})

df_combined.to_csv('explanation.csv', index=False)